In [ ]:
import matplotlib.patches as mpatches
import numpy as np
import os
import math
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
params = {
   'axes.labelsize': 12,
   'font.size': 10,
   'legend.fontsize': 11,
   'xtick.labelsize': 10,
   'ytick.labelsize': 12,
   'text.usetex': False,
   'figure.figsize': [10, 2] # instead of 4.5, 4.5
   }
plt.rcParams.update(params)

## helper function

In [ ]:
def autolabel(rects, ses):
    # attach some text labels
    for rect, se in zip(rects, ses):
        height = rect.get_height()
        plt.axes().text(rect.get_x()+rect.get_width()/2., height+1.03*se, '{:2.3f}'.format(float(height)).lstrip('0'),
                ha='center', va='bottom',fontsize=10)

## Fixed Parameter

In [ ]:
# experiment
exp= 'different_TR'
exp_label = 'Time Segment Matching using Different Number of TRs'
model = 'multi_srm'
portion = [0.2,0.4,0.6,0.8]
port_name = ['$20$','$40$','$60$','$80$','$100$']

# dictionaries
cl_dict = {0:1/225,1:2/297,2:2/297,3:2/1973} # chance level of each dataset
feat_dict = {'multi_srm':[75,75,100]}
roi_dict = {'dmn':0,'pt':1,'eac':2}
# change multi_srm name!
ds_dict = {0:'GreenEyes',1:'Milky',2:'Vodka',3:'Sherlock'}

# paths 
input_full = '../../output/accu_bar/mysseg_all/{}/{}_ds{}.npz' #roi,model,ds
input_file = '../../output/accu_bar/{}/{}/{}_ds{}_port{}.npz' #exp,roi,model,ds,port
output_path = '../../output/figures/{}/' #exp
if not os.path.exists(output_path.format(exp)):
    os.makedirs(output_path.format(exp))
output_file = output_path+'{}_ds{}' #roi,ds

## Plotting Parameter

In [ ]:
roi_all = ['dmn','pt','eac']
max_accu = 0.75
datasets = [0,3,2,1]

## Aggregate Accuracy

In [ ]:
all_mean = [] # length # of roi
all_se = [] # length # of roi
# aggregate each roi
for roi in roi_all:
    roi_mean = []
    roi_se = []
    for ds in datasets:
        # portion part
        for port in portion:
            ws=np.load(input_file.format(exp,roi,model,ds,port))
            roi_mean.append(ws['mean'].item())
            roi_se.append(ws['se'].item())
        # full sec. datasets
        ws = np.load(input_full.format(roi,model,ds))
        roi_mean.append(ws['mean'].item())
        roi_se.append(ws['se'].item())
    all_mean.append(roi_mean)
    all_se.append(roi_se)

## Plot all roi

In [ ]:
width=1
nmodel = len(portion)+1
ndata = len(datasets)
group_width = 1.25*width*nmodel
center_all = np.linspace(0,group_width*(ndata-1),ndata)
# set colors
color = 'dodgerblue'

for m,roi in enumerate(roi_all):
    ds_idx = []
    ds_name = []
    xtick_idx = []
    xtick_name = []
    plt.figure()
    # configure bars
    for i,(ds, center) in enumerate(zip(datasets,center_all)):
        mean = list(all_mean[m][i*nmodel:(i+1)*nmodel])
        se   = list(all_se[m][i*nmodel:(i+1)*nmodel])    
        idx = np.arange(center,center+nmodel*width,width)
        error_config = {'ecolor': '0','capsize':3}   
        rects = plt.bar(idx, mean, yerr=se, align='center', error_kw=error_config, width = width-0.1)
        # set colors
        for rect_i in range(len(rects)):
            rects[rect_i].set_color(color)
        autolabel(rects, se)
        ds_idx.append(center+(nmodel-1)*width/2)        
        ds_name.append(ds_dict[ds]) 
        xtick_idx.extend(idx)
        xtick_name.extend(port_name)
    plt.xticks(xtick_idx,xtick_name)
    
    plt.ylabel('Accuracy')
    plt.xlabel('Percentage of number of TRs used in sec. datasets')
    left_lim = center_all[0]-0.5*width-0.5
    right_lim = center_all[-1]+(nmodel-0.5)*width+0.5
    plt.xlim([left_lim,right_lim])
    plt.ylim([0,max_accu])

    # plot chance accuracy of each dataset
    for d,center in enumerate(center_all):
        cl = cl_dict[datasets[d]]
        line = plt.plot([center-width, center+(nmodel+1)*width], [cl, cl], 'k-.', linewidth=2)
       
    # Add texts
    plt.text((right_lim+left_lim)/2, 1.02*max_accu, exp_label+' (ROI: '+roi.upper()+')',fontsize=12,horizontalalignment='center', verticalalignment='bottom')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    plt.text(left_lim+0.81*(right_lim-left_lim), 0.78*max_accu, 'k = '+str(feat_dict[model][roi_dict[roi]]),bbox=props,fontsize=11,horizontalalignment='center', verticalalignment='top')
    for d in range(len(datasets)):
        plt.text(ds_idx[d],0.95*max_accu,'Prm.: '+ds_name[d],bbox=props,fontsize=11,horizontalalignment='center', verticalalignment='top')
    
    # legend
    plt.legend(line ,['chance'],bbox_to_anchor=(0.,.87,1.005,0.),loc=1,ncol=1)

            
#     plt.savefig(output_file.format(exp,roi,datasets).replace(' ','')+'.eps', format='eps', dpi=200,bbox_inches='tight')
    plt.savefig(output_file.format(exp,roi,datasets).replace(' ','')+'.pdf', format='pdf', dpi=200,bbox_inches='tight')